In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
import sys

from PIL import Image
from tensorflow import keras

In [ ]:
import os
import glob

In [ ]:
path = 'C:/yewon/pythonAI/data/datasets'
                                                              

path_angry = path+ '/angry'
path_disgust = path + '/disgust'
path_fear = path+ '/fear'
path_happy = path+ '/happy'
path_neutral = path+ '/neutral'
path_sad = path+ '/sad'
path_surprise = path+ '/surprise'


angry = glob.glob(path_angry+ '/*')
disgust = glob.glob(path_disgust+ '/*')
fear = glob.glob(path_fear+ '/*')
happy = glob.glob(path_happy+ '/*')
neutral = glob.glob(path_neutral+ '/*')
sad = glob.glob(path_sad+ '/*')
surprise = glob.glob(path_surprise+ '/*')

print(angry[0])

In [ ]:
def rename(files):
    if 'angry' in files[0]: 
        for i,f in enumerate(files): # i: 순번, f: 파일
            os.rename(f, os.path.join(path+"/angry", 'angry_' + '{0:03d}.jpg'.format(i)))
        angry = glob.glob(path+"/angry" + '/*')    
        print("angry {}번째 이미지까지 성공".format(i+1))
    
    elif 'disgust' in files[0]:
        for i,f in enumerate(files):
            os.rename(f, os.path.join(path+"/disgust", 'disgust_' + '{0:03d}.jpg'.format(i)))
        disgust = glob.glob(path+"/disgust"+'/*')
        print("disgust {}번째 이미지까지 성공".format(i+1))
    
    elif 'fear' in files[0]:
        for i,f in enumerate(files):
            os.rename(f, os.path.join(path+"/fear", 'fear_' + '{0:03d}.jpg'.format(i)))
        fear = glob.glob(path+"/fear"+'/*')
        print("fear {}번째 이미지까지 성공".format(i+1))
    
    elif 'happy' in files[0]:
        for i,f in enumerate(files):
            os.rename(f, os.path.join(path+"/happy", 'happy_' + '{0:03d}.jpg'.format(i)))
        happy = glob.glob(path+"/happy"+'/*')
        print("happy {}번째 이미지까지 성공".format(i+1))
    
    elif 'neutral' in files[0]:
        for i,f in enumerate(files):
            os.rename(f, os.path.join(path+"/neutral", 'neutral_' + '{0:03d}.jpg'.format(i)))
        neutral = glob.glob(path+"/neutral"+'/*')
        print("neutral {}번째 이미지까지 성공".format(i+1))
    
    elif 'sad' in files[0]:
        for i,f in enumerate(files):
            os.rename(f, os.path.join(path+"/sad", 'sad_' + '{0:03d}.jpg'.format(i)))
        sad = glob.glob(path+"/sad"+'/*')
        print("sad {}번째 이미지까지 성공".format(i+1))
    
    elif 'surprise' in files[0]:
        for i,f in enumerate(files):
            os.rename(f, os.path.join(path+"/surprise", 'surprise_' + '{0:03d}.jpg'.format(i)))
        surprise = glob.glob(path+"/surprise"+'/*')
        print("surprise {}번째 이미지까지 성공".format(i+1))

In [ ]:
angry = sorted(glob.glob(path+"/angry" + '/*'))
disgust = sorted(glob.glob(path+"/disgust"+'/*'))
fear = sorted(glob.glob(path+"/fear"+'/*'))
happy = sorted(glob.glob(path+"/happy"+'/*'))
neutral = sorted(glob.glob(path+"/neutral"+'/*'))
sad = sorted(glob.glob(path+"/sad"+'/*'))
surprise = sorted(glob.glob(path+"/surprise"+'/*'))

In [ ]:
print('angry 이미지 개수: {}'.format(len(angry)))
print('disgust 이미지 개수: {}'.format(len(disgust)))
print('fear 이미지 개수: {}'.format(len(fear)))
print('happy 이미지 개수: {}'.format(len(happy)))
print('neutral 이미지 개수: {}'.format(len(neutral)))
print('sad 이미지 개수: {}'.format(len(sad)))
print('surprise 이미지 개수: {}'.format(len(surprise)))

In [ ]:
feeling_images = [] 

def put_label(feeling,emotion):
  
  for i in range(len(feeling)):
    filename = str(feeling[i]).split('/')[-1] 
    filename = filename.replace('.jpg','')

    feeling_images.append([filename,str(emotion)])  

In [ ]:
put_label(angry,'angry')
put_label(disgust,'disgust')
put_label(fear,'fear')
put_label(happy,'happy')
put_label(neutral,'neutral')
put_label(sad,'sad')
put_label(surprise,'surprise')

In [ ]:
colnames = ['file_name','emotion']  

feelings_df= pd.DataFrame(feeling_images,columns = colnames)
feelings_df

# 4-1. Stratified K Fold

In [ ]:
from sklearn.model_selection import StratifiedKFold 

In [ ]:
skf = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 42) 

feelings_df['fold'] = -1 

for idx, (t,v) in enumerate(skf.split(feelings_df,feelings_df['emotion']),1):  
    print(t,v,len(v))
    feelings_df.loc[v,'fold'] = idx

In [ ]:
value_counts = feelings_df[feelings_df['fold'] != 5]['emotion'].value_counts().sort_index()

plt.bar(range(len(value_counts)),value_counts.values,align = "center")
plt.xticks(range(len(value_counts)),value_counts.index.values)

plt.tight_layout()
plt.show()

In [ ]:
feelings_df.to_csv('C:/yewon/pythonAI/data/feelings.csv',index = False)

# train test 나누기

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(         
    feelings_df.drop('emotion',1),feelings_df['emotion'])  

In [ ]:
len(x_train)

In [ ]:
len(x_test)

In [ ]:
print(x_train.shape, y_train.shape)

In [ ]:
print(x_test.shape, y_test.shape)

In [ ]:
feelings_df.dtypes

In [ ]:
# 망함 모르겠음
# feelings_df 데이터 타입을 float으로 바꾸려 했음
feelings = feelings_df.apply(pd.to_numeric)
feelings

In [ ]:
# 위에 안돼서 얘도 안됨

# 데이터 전처리
x_train = x_train.astype(np.float32) / 255.
x_test = x_train.astype(np.float32) / 255.

# one-hot encoding
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(
    buffer_size = 100000).batch(64)
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(64)

In [ ]:
imgs, lbs = next(iter(train_dataset))
print(f'Feature batch shape : {imgs.shape}')
print(f'Label batch shape : {lbs.shape}')

img = imgs[0]
lb = lbs[0]
plt.imshow(img, cmap='gray')
plt.show()
print(f'Label : {lb}')

In [ ]:
a = np.arange(10)
print(a)

ds_tensors = tf.data.Dataset.from_tensor_slices(a)
print(ds_tensors)

for x in ds_tensors:
  print(x)

In [ ]:
ds_tensors = ds_tensors.map(tf.square).shuffle(10).batch(2)

In [ ]:
for _ in range(3):
  for x in ds_tensors:
    print(x)

# 모델 만들기

In [ ]:
def create_seq_model():
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28,28)))
  model.add(keras.layers.Dense(128, activation='relu'))
  model.add(keras.layers.Dropout(0.2))
  model.add(keras.layers.Dense(10, activation='softmax'))
  return model

In [ ]:
def create_func_model():
  inputs = keras.Input(shape=(28, 28))
  flatten = keras.layers.Flatten()(inputs)
  dense = keras.layers.Dense(128, activation='relu')(flatten)
  drop = keras.layers.Dropout(0.2)(dense)
  outputs = keras.layers.Dense(10, activation='softmax')(drop)
  model = keras.Model(inputs=inputs, outputs=outputs)
  return model

In [ ]:
class SubClassModel(keras.Model):
  def __init__(self):
    super(SubClassModel, self).__init__()
    self.flatten = keras.layers.Flatten(input_shape=(28, 28))
    self.dense1 = keras.layers.Dense(128, activation='relu')
    self.drop = keras.layers.Dropout(0.2)
    self.dense2 = keras.layers.Dense(10, activation='softmax')
  def call(self, x, training=False):
    x = self.flatten(x)
    x = self.dense1(x)
    x = self.drop(x)
    return self.dense2(x)

In [ ]:
inputs = tf.zeros((1, 28, 28))
subclass_model(inputs)
subclass_model.summary()

In [ ]:
learning_rate = 0.001
seq_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

In [ ]:
history = seq_model.fit(train_dataset, epochs=10, validation_data=test_dataset)

In [ ]:
plt.plot(history.history['loss'], 'b-', label='loss')
plt.plot(history.history['val_loss'], 'r--', label='val_loss')
plt.xlabel('Epoch')
plt.legend()
plt.show()

In [ ]:
plt.plot(history.history['accuracy'], 'b-', label='acc') 
plt.plot(history.history['val_accuracy'], 'r--', label='val_acc') 
plt.xlabel('Epoch')
plt.legend()
plt.show()

# segmentation 해결 방법??

In [ ]:
import pixellib
from pixellib.semantic import semantic_segmentation

segment_image = semantic_segmentation()
segment_image.load_pascalvoc_model("deeplabv3_xception_tf_dim_ordering_tf_kernels.h5") 
segment_image.segmentAsPascalvoc("angry_000.jpg", output_image_name = "angry_000new.jpg")

In [ ]:
image_dir = "이름바꾸기"

In [ ]:
image_files = glob(image_dir + "*.jpg")
len(image_files)

In [ ]:
import cv2
import xml.etree.ElementTree as et
from matplotlib.patches import Rectangle

In [ ]:
images_path = image_files[60]
bbox_path = images_path.replace(image_dir, bbox_dir).replace('jpg', 'xml')

image = cv2.imread(images_path) # 이미지 파일을 읽어옴, 절대 또는 상대경로 사용
# image.shape : (200, 222, 3)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # BlueGreenRed , RedGreenBlue 

print(bbox_path)
tree = et.parse(bbox_path)

xmin = float(tree.find('./object/bndbox/xmin').text)
xmax = float(tree.find('./object/bndbox/xmax').text)
ymin = float(tree.find('./object/bndbox/ymin').text)
ymax = float(tree.find('./object/bndbox/ymax').text)

rect_x = xmin
rect_y = ymin
rect_w = xmax - xmin
rect_h = ymax - ymin

rect = Rectangle((rect_x, rect_y), rect_w, rect_h, fill=False, color="red")
plt.axes().add_patch(rect)
plt.imshow(image)
plt.show()

In [ ]:
image_path = image_files[60]
seg_path = image_path.replace(image_dir, seg_dir).replace('jpg', 'png')

image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
seg_map = cv2.imread(seg_path, cv2.IMREAD_GRAYSCALE)

plt.figure(figsize=(15, 15))
plt.subplot(1, 2, 1)
plt.imshow(image)
plt.subplot(1, 2, 2)
plt.imshow(seg_map)

plt.show()